In [2]:
pip install torch

   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/109.3 MB 2.0 MB/s eta 0:00:56
   ---------------------------------------- 0.2/109.3 MB 2.9 MB/s eta 0:00:38
   ---------------------------------------- 0.4/109.3 MB 3.1 MB/s eta 0:00:35
   ---------------------------------------- 0.5/109.3 MB 2.9 MB/s eta 0:00:38
   ---------------------------------------- 0.6/109.3 MB 2.7 MB/s eta 0:00:41
   ---------------------------------------- 0.7/109.3 MB 2.7 MB/s eta 0:00:41
   ---------------------------------------- 0.8/109.3 MB 2.6 MB/s eta 0:00:43
   ---------------------------------------- 0.9/109.3 MB 2.6 MB/s eta 0:00:42
   ---------------------------------------- 1.1/109.3 MB 2.6 MB/s eta 0:00:43
   ---------------------------------------- 1.1/109.3 MB 2.5 MB/s eta 0:00:44
   ---------------------------------------- 1.2/109.3 MB 2.5 MB/s eta 0:00:44
   ---------------------------------------- 1.3/109.3 MB 2.4 MB/s eta 0

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'c:\\Python312\\share'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

In [37]:
player_stats = pd.read_csv('strict_player_stats.csv')
fifa_data = pd.read_csv('strict_fifa_data.csv')
validation_df = pd.read_csv('strict_validation.csv')

# Zusammenführen
merged_data = pd.merge(
    player_stats, 
    validation_df[['Stats_Player', 'FIFA_Overall']], 
    left_on='Player', 
    right_on='Stats_Player',
    how='inner'
)

In [38]:
def prepare_features_for_training(df, exclude_cols=None):
    """Bereitet Features für ML vor"""
    if exclude_cols is None:
        exclude_cols = ['Rk', 'Born', 'MP', 'Starts', 'Min', '90s', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age']
    
    # Numerische Features
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feature_cols = [col for col in numeric_cols if col not in exclude_cols and df[col].nunique() > 1]
    
    # Daten bereinigen
    features_df = df[feature_cols].copy()
    features_df = features_df.fillna(features_df.median())
    
    # Outlier behandeln
    for col in features_df.columns:
        Q1 = features_df[col].quantile(0.25)
        Q3 = features_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        features_df[col] = np.clip(features_df[col], lower_bound, upper_bound)
    
    # Skalieren
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(features_df)
    
    return X_scaled, scaler, features_df.columns.tolist(), features_df

# Features vorbereiten
X_scaled, feature_scaler, feature_names, features_df = prepare_features_for_training(merged_data)
y_target = merged_data['FIFA_Overall']

print(f"🎯 Features: {len(feature_names)}")
print(f"🔢 Target Range: {y_target.min()} - {y_target.max()}")

🎯 Features: 132
🔢 Target Range: 47 - 91


In [39]:
# Optimale Cluster-Anzahl finden
def find_optimal_clusters(X, max_k=10):
    inertias = []
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    return inertias

inertias = find_optimal_clusters(X_scaled)
optimal_k = 6  # Basierend auf Elbow-Methode

# K-Means Clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)
merged_data['Cluster'] = cluster_labels

# PCA für Visualisierung
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
merged_data['PC1'] = X_pca[:, 0]
merged_data['PC2'] = X_pca[:, 1]

print(f"📈 Clustering abgeschlossen: {optimal_k} Cluster")

📈 Clustering abgeschlossen: 6 Cluster


In [40]:
class PlayerRatingPredictor(nn.Module):
    def __init__(self, input_size):
        super(PlayerRatingPredictor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # Output zwischen 0-1
        )
    
    def forward(self, x):
        output = self.network(x)
        return output * 100  # Skaliere auf 0-100

def train_pytorch_model(X, y):
    # Target auf 0-1 skalieren für Training
    y_scaled = y / 100.0
    
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.FloatTensor(y_scaled.values).view(-1, 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
    
    model = PlayerRatingPredictor(X.shape[1])
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    losses = []
    print("🧠 Training PyTorch Model...")
    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train) / 100  # Zurück zu 0-1 für Loss-Berechnung
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
        if epoch % 200 == 0:
            print(f'  Epoch {epoch}: Loss = {loss.item():.4f}')
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train)
        test_pred = model(X_test)
        
        # R² auf original Skala berechnen
        train_r2 = r2_score(y_train.numpy().flatten() * 100, train_pred.numpy().flatten())
        test_r2 = r2_score(y_test.numpy().flatten() * 100, test_pred.numpy().flatten())
    
    print(f"✅ PyTorch NN: Train R² = {train_r2:.3f}, Test R² = {test_r2:.3f}")
    return model, losses

# Model neu trainieren
pytorch_model, losses = train_pytorch_model(X_scaled, y_target)

🧠 Training PyTorch Model...
  Epoch 0: Loss = 0.0586
  Epoch 200: Loss = 0.0007
  Epoch 400: Loss = 0.0003
  Epoch 600: Loss = 0.0002
  Epoch 800: Loss = 0.0002
✅ PyTorch NN: Train R² = 0.985, Test R² = 0.955


In [41]:
# Random Forest als Vergleich
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y_target)
rf_predictions = rf_model.predict(X_scaled)
rf_r2 = r2_score(y_target, rf_predictions)
rf_mae = mean_absolute_error(y_target, rf_predictions)

print(f"🌲 Random Forest: R² = {rf_r2:.3f}, MAE = {rf_mae:.2f}")

🌲 Random Forest: R² = 1.000, MAE = 0.02


In [42]:
def make_predictions(model, player_data, feature_names, scaler, model_type='pytorch'):
    """Macht Vorhersagen für alle Spieler (garantiert 0-100)"""
    X_new = player_data[feature_names].copy().fillna(player_data[feature_names].median())
    X_scaled = scaler.transform(X_new)
    
    if model_type == 'pytorch':
        model.eval()
        X_tensor = torch.FloatTensor(X_scaled)
        
        with torch.no_grad():
            predictions = model(X_tensor).numpy().flatten()
            # Sicherstellen dass Werte zwischen 0-100 sind
            predictions = np.clip(predictions, 0, 100)
    else:  # random forest
        predictions = model.predict(X_scaled)
        predictions = np.clip(predictions, 0, 100)
    
    return predictions

# Vorhersagen erstellen
pytorch_predictions = make_predictions(pytorch_model, merged_data, feature_names, feature_scaler, 'pytorch')
rf_predictions = make_predictions(rf_model, merged_data, feature_names, feature_scaler, 'rf')

# Ergebnisse zusammenführen
results_df = merged_data[['Player', 'Nation', 'Pos', 'Squad', 'Age', 'FIFA_Overall', 'Cluster']].copy()
results_df['Predicted_Rating_PyTorch'] = pytorch_predictions
results_df['Predicted_Rating_RF'] = rf_predictions
results_df['Difference_PyTorch'] = results_df['Predicted_Rating_PyTorch'] - results_df['FIFA_Overall']
results_df['Difference_RF'] = results_df['Predicted_Rating_RF'] - results_df['FIFA_Overall']

print("✅ Alle Vorhersagen erstellt (0-100 garantiert)")

✅ Alle Vorhersagen erstellt (0-100 garantiert)


In [43]:
# Validierung der Ratings
print("\n" + "="*50)
print("🔍 VALIDIERUNG DER RATINGS (0-100)")
print("="*50)

print(f"\n🎯 PyTorch Model:")
print(f"   Min:    {results_df['Predicted_Rating_PyTorch'].min():.1f}")
print(f"   Max:    {results_df['Predicted_Rating_PyTorch'].max():.1f}")
print(f"   Mean:   {results_df['Predicted_Rating_PyTorch'].mean():.1f}")
print(f"   Std:    {results_df['Predicted_Rating_PyTorch'].std():.1f}")

print(f"\n🌲 Random Forest Model:")
print(f"   Min:    {results_df['Predicted_Rating_RF'].min():.1f}")
print(f"   Max:    {results_df['Predicted_Rating_RF'].max():.1f}")
print(f"   Mean:   {results_df['Predicted_Rating_RF'].mean():.1f}")
print(f"   Std:    {results_df['Predicted_Rating_RF'].std():.1f}")

print(f"\n📊 FIFA Overall (Original):")
print(f"   Min:    {results_df['FIFA_Overall'].min():.1f}")
print(f"   Max:    {results_df['FIFA_Overall'].max():.1f}")
print(f"   Mean:   {results_df['FIFA_Overall'].mean():.1f}")


🔍 VALIDIERUNG DER RATINGS (0-100)

🎯 PyTorch Model:
   Min:    48.9
   Max:    100.0
   Mean:   76.7
   Std:    6.9

🌲 Random Forest Model:
   Min:    50.0
   Max:    90.8
   Mean:   73.8
   Std:    6.6

📊 FIFA Overall (Original):
   Min:    47.0
   Max:    91.0
   Mean:   73.9


In [55]:
# Nach verschiedenen Kriterien sortieren
print(f"\n🏆 TOP 20 SPIELER NACH VORHERSAGE:")
top_players = results_df.nlargest(20, 'Predicted_Rating_PyTorch')[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]
print(top_players.round(1))

print(f"\n🔍 INTERESSANTE FÄLLE (Große Unterschiede):")
# Größte positive Unterschiede (Underrated)
underrated = results_df.nlargest(10, 'Difference_PyTorch')[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]
print("Underrated (Model > FIFA):")
print(underrated.round(1))

# Größte negative Unterschiede (Overrated)  
overrated = results_df.nsmallest(10, 'Difference_PyTorch')[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]
print("\nOverrated (FIFA > Model):")
print(overrated.round(1))


🏆 TOP 20 SPIELER NACH VORHERSAGE:
                   Player   Pos                  Squad  FIFA_Overall  \
1453          Pedro Ortiz    MF                Sevilla            63   
1568       Anwar El Ghazi    FW  Everton / Aston Villa            75   
481             Josh Maja    FW               Bordeaux            71   
845         Neco Williams    DF              Liverpool            71   
468       Zidan Sertdemir  MFFW             Leverkusen            62   
64           Kaide Gordon    FW              Liverpool            62   
1620         Keinan Davis    FW            Aston Villa            72   
145            Liam Delap    FW        Manchester City            66   
1055           Kasim Nuhu    DF             Hoffenheim            64   
1340               Blanco    MF            Real Madrid            71   
1013          Brain Oddei    FW               Sassuolo            64   
188           Jacopo Dezi    MF                Venezia            69   
1351  Josï¿½ Luis Morales  FW

In [56]:
results_df[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]

,Player,Pos,Squad,FIFA_Overall,Predicted_Rating_PyTorch,Difference_PyTorch
0,Patrick Cutrone,FW,Empoli,72,71.685013,-0.314987
1,Aleksandr Kokorin,FWMF,Fiorentina,72,73.218727,1.218727
2,David Raya,GK,Brentford,76,81.855118,5.855118
3,Jason Denayer,DF,Lyon,77,75.873749,-1.126251
4,Hannes Wolf,FWMF,M'Gladbach,72,75.538605,3.538605
5,Marius Wolf,DFFW,Dortmund,77,76.930542,-0.069458
6,Francisco Sierralta,DF,Watford,72,71.762306,-0.237694
7,Luca Vignali,DF,Spezia,69,74.215630,5.215630
8,Javier Manquillo,DF,Newcastle Utd,74,75.265434,1.265434
9,Oihan Sancet,FW,Athletic Club,76,75.890816,-0.109184


In [46]:
# Detaillierte Cluster-Analyse
print(f"\n📈 DETAILLIERTE CLUSTER-ANALYSE:")
cluster_details = results_df.groupby('Cluster').agg({
    'Player': 'count',
    'FIFA_Overall': ['min', 'max', 'mean'],
    'Predicted_Rating_PyTorch': ['min', 'max', 'mean'],
    'Age': 'mean',
    'Pos': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Mixed'
}).round(1)

cluster_details.columns = ['Anzahl', 'FIFA_Min', 'FIFA_Max', 'FIFA_Mean', 
                          'Pred_Min', 'Pred_Max', 'Pred_Mean', 'Age_Mean', 'Haupt_Position']
print(cluster_details)

# Beispiel-Spieler pro Cluster
print(f"\n👥 BEISPIEL-SPIELER PRO CLUSTER:")
for cluster in sorted(results_df['Cluster'].unique()):
    cluster_players = results_df[results_df['Cluster'] == cluster]
    top_player = cluster_players.nlargest(1, 'Predicted_Rating_PyTorch').iloc[0]
    avg_player = cluster_players.nlargest(5, 'Predicted_Rating_PyTorch').iloc[2]  # Mittlerer Spieler
    
    print(f"\nCluster {cluster}:")
    print(f"  Top:    {top_player['Player']} ({top_player['Pos']}) - {top_player['Predicted_Rating_PyTorch']:.1f}")
    print(f"  Avg:    {avg_player['Player']} ({avg_player['Pos']}) - {avg_player['Predicted_Rating_PyTorch']:.1f}")
    print(f"  Größe:  {len(cluster_players)} Spieler")


📈 DETAILLIERTE CLUSTER-ANALYSE:
         Anzahl  FIFA_Min  FIFA_Max  FIFA_Mean   Pred_Min    Pred_Max  \
Cluster                                                                 
0           431        52        84       72.7  53.500000   99.500000   
1           318        50        91       72.2  51.200001   99.400002   
2           317        47        90       73.9  48.900002  100.000000   
3           252        55        91       77.3  63.000000   99.800003   
4           168        57        90       72.4  60.099998   98.900002   
5           198        61        88       75.8  65.800003   99.900002   

         Pred_Mean  Age_Mean Haupt_Position  
Cluster                                      
0        75.300003      25.3             DF  
1        74.300003      25.0             FW  
2        74.900002      26.0             DF  
3        81.000000      25.6           FWMF  
4        76.900002      26.3             GK  
5        80.699997      25.8             DF  

👥 BEISPIEL-SP